# Attentional Networks in Computer Vision
Prepared by comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [ ]:
PYTORCH_ENABLE_MPS_FALLBACK=1

In [ ]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=False,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=False,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=False,
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment.

In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf

In [ ]:
class SelfAttention(nn.Module):

    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()

        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads

        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token
        ## - D corresponds to model dimensionality
        b,n,d = x.shape

        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)

        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b,n,self.num_heads,self.head_dims)).permute(0,2,1,3), [q_,k_,v_])

        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        ## Compute attention logits. Note that this operation is conducted as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
#         print(f"input shape: {q.shape}")
#         print(f"output shape:{b} {q.shape[1]} {n} {n}")
#         print(f"{k.transpose(-2, -1).shape}")
        attention_logits = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dims ** 0.5)

        ## Compute attention Weights. Note that this operation is conducted as a
        ## Softmax Normalization across the keys dimension.
        ## Hint: You can apply the Softmax operation across the final dimension

        attention_weights = F.softmax(attention_logits, dim=-1)


        ## Compute output values. Note that this operation is conducted as a
        ## batched matrix multiplication between the Attention Weights matrix and
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order

        attn_out = torch.matmul(attention_weights, v).permute(0,2,1,3).reshape(b,n,self.proj_dims)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE
        ################################################################################

        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [ ]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 32
    layer = SelfAttention(32,64,4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [ ]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()

        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        o = F.relu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [ ]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
## input --> layernorm --> SelfAttention --> skip connection
##       --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()

###############################################################
# TODO: Complete the consturctor of  TransformerBlock module  #
###############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        self.layer_norm1 = nn.LayerNorm(hidden_dims)
        self.self_attention = SelfAttention(input_dims=hidden_dims, head_dims=(hidden_dims//num_heads), num_heads=num_heads, bias=False)
        self.layer_norm2 = nn.LayerNorm(hidden_dims)
        self.mlp = MLP(hidden_dims, hidden_dims*4, hidden_dims, bias=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #
###################################################################

    def forward(self, x):

##############################################################
# TODO: Complete the forward of TransformerBlock module      #
##############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****
        input1 = x
        x = self.layer_norm1(x)
        x = self.self_attention(x)
        x = self.mlp(x + input1)

        input2 = x
        x = self.layer_norm2(x)
        x = self.mlp(x)
        x = x + input2
        return x

 # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #
###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 128).

In [ ]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128,4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size())
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [ ]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()

        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k

        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k

        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width

        ## Declare module's parameters

        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)

        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))

        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])

        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)

        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)

    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n

    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)

        ## retrieve class token
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)

        ## add positional embedding to input patches + class token
        xcls_pos_embed = xcls_embed + self.pos_embedding

        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)

        ## select the class token
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))

        ## create output
        out = self.out_mlp(out_cls_token)

        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 10).

In [ ]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size())
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network.

The check_batch_accuracy function is provided for you below:

In [ ]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1)
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [ ]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch

    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches

    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU

        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()

        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets)
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [ ]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch

    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches

    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)

            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free.

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`.

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [ ]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [ ]:
learning_rate = 0.002
regularization_val = 1e-6
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

model = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims=hidden_dims,
            input_dims=input_dims,
            output_dims=output_dims,
            num_trans_layers=num_trans_layers,
            num_heads=num_heads,
            image_k=image_k,
            patch_k=patch_k,
            bias=False).to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(15):
    tr_acc = train(network, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))

    eval_acc = evaluate(network, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)

print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 3.247 | Acc: 4.000% (1/25)
Loss: 4.329 | Acc: 2.000% (1/50)
Loss: 4.994 | Acc: 8.000% (6/75)
Loss: 5.377 | Acc: 7.000% (7/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 7.0

---- Evaluation in process ----
Loss: 3.790 | Acc: 20.000% (5/25)
Loss: 3.942 | Acc: 14.000% (7/50)
Loss: 4.192 | Acc: 10.667% (8/75)
Loss: 4.266 | Acc: 11.000% (11/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 11.0

Epoch: 1
Loss: 4.333 | Acc: 20.000% (5/25)
Loss: 4.028 | Acc: 16.000% (8/50)
Loss: 3.921 | Acc: 16.000% (12/75)
Loss: 3.731 | Acc: 16.000% (16/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 16.0

---- Evaluation in process ----
Loss: 3.128 | Acc: 12.000% (3/25)
Loss: 3.080 | Acc: 18.000% (9/50)
Loss: 3.164 | Acc: 13.333% (10/75)
Loss: 3.122 | Acc: 12.000% (12/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 12.0

Epoch: 2
Loss: 3.077 | Acc: 12.000% (3/25)
Loss: 2.959 | 

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [ ]:
learning_rate = 0.002
regularization_val = 1e-6
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims=hidden_dims,
            input_dims=input_dims,
            output_dims=output_dims,
            num_trans_layers=num_trans_layers,
            num_heads=num_heads,
            image_k=image_k,
            patch_k=patch_k,
            bias=False).to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0)


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))

    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))

    tr_accs.append(tr_acc)
    test_accs.append(test_acc)

print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0
Loss: 3.203 | Acc: 6.250% (4/64)
Loss: 3.842 | Acc: 10.156% (13/128)
Loss: 3.785 | Acc: 9.896% (19/192)
Loss: 5.399 | Acc: 8.984% (23/256)
Loss: 5.111 | Acc: 8.750% (28/320)
Loss: 4.906 | Acc: 9.635% (37/384)
Loss: 4.679 | Acc: 10.268% (46/448)
Loss: 4.514 | Acc: 9.570% (49/512)
Loss: 4.535 | Acc: 9.375% (54/576)
Loss: 4.548 | Acc: 9.219% (59/640)
Loss: 4.430 | Acc: 9.233% (65/704)
Loss: 4.282 | Acc: 8.984% (69/768)
Loss: 4.153 | Acc: 9.255% (77/832)
Loss: 4.049 | Acc: 9.598% (86/896)
Loss: 4.020 | Acc: 9.375% (90/960)
Loss: 3.949 | Acc: 9.473% (97/1024)
Loss: 3.902 | Acc: 9.651% (105/1088)
Loss: 3.856 | Acc: 9.896% (114/1152)
Loss: 3.813 | Acc: 10.033% (122/1216)
Loss: 3.749 | Acc: 10.156% (130/1280)
Loss: 3.691 | Acc: 10.045% (135/1344)
Loss: 3.627 | Acc: 10.369% (146/1408)
Loss: 3.573 | Acc: 10.462% (154/1472)
Loss: 3.531 | Acc: 10.482% (161/1536)
Loss: 3.489 | Acc: 10.625% (170/1600)
Loss: 3.446 | Acc: 10.637% (177/1664)
Loss: 3.410 | Acc: 10.532% (182/1728)
Loss: 3.377 |

Loss: 2.275 | Acc: 19.940% (2782/13952)
Loss: 2.274 | Acc: 19.991% (2802/14016)
Loss: 2.272 | Acc: 20.014% (2818/14080)
Loss: 2.271 | Acc: 20.044% (2835/14144)
Loss: 2.270 | Acc: 20.080% (2853/14208)
Loss: 2.269 | Acc: 20.116% (2871/14272)
Loss: 2.269 | Acc: 20.096% (2881/14336)
Loss: 2.268 | Acc: 20.111% (2896/14400)
Loss: 2.267 | Acc: 20.147% (2914/14464)
Loss: 2.266 | Acc: 20.168% (2930/14528)
Loss: 2.265 | Acc: 20.169% (2943/14592)
Loss: 2.264 | Acc: 20.183% (2958/14656)
Loss: 2.263 | Acc: 20.224% (2977/14720)
Loss: 2.261 | Acc: 20.258% (2995/14784)
Loss: 2.259 | Acc: 20.299% (3014/14848)
Loss: 2.259 | Acc: 20.312% (3029/14912)
Loss: 2.257 | Acc: 20.346% (3047/14976)
Loss: 2.256 | Acc: 20.399% (3068/15040)
Loss: 2.255 | Acc: 20.399% (3081/15104)
Loss: 2.254 | Acc: 20.425% (3098/15168)
Loss: 2.253 | Acc: 20.424% (3111/15232)
Loss: 2.252 | Acc: 20.437% (3126/15296)
Loss: 2.251 | Acc: 20.482% (3146/15360)
Loss: 2.249 | Acc: 20.539% (3168/15424)
Loss: 2.248 | Acc: 20.558% (3184/15488)


Loss: 2.107 | Acc: 23.676% (6440/27200)
Loss: 2.106 | Acc: 23.669% (6453/27264)
Loss: 2.106 | Acc: 23.701% (6477/27328)
Loss: 2.105 | Acc: 23.708% (6494/27392)
Loss: 2.105 | Acc: 23.736% (6517/27456)
Loss: 2.105 | Acc: 23.721% (6528/27520)
Loss: 2.104 | Acc: 23.731% (6546/27584)
Loss: 2.103 | Acc: 23.781% (6575/27648)
Loss: 2.103 | Acc: 23.777% (6589/27712)
Loss: 2.102 | Acc: 23.794% (6609/27776)
Loss: 2.102 | Acc: 23.822% (6632/27840)
Loss: 2.101 | Acc: 23.853% (6656/27904)
Loss: 2.101 | Acc: 23.849% (6670/27968)
Loss: 2.101 | Acc: 23.862% (6689/28032)
Loss: 2.100 | Acc: 23.872% (6707/28096)
Loss: 2.100 | Acc: 23.874% (6723/28160)
Loss: 2.099 | Acc: 23.887% (6742/28224)
Loss: 2.099 | Acc: 23.883% (6756/28288)
Loss: 2.099 | Acc: 23.889% (6773/28352)
Loss: 2.098 | Acc: 23.906% (6793/28416)
Loss: 2.097 | Acc: 23.922% (6813/28480)
Loss: 2.097 | Acc: 23.900% (6822/28544)
Loss: 2.097 | Acc: 23.899% (6837/28608)
Loss: 2.096 | Acc: 23.926% (6860/28672)
Loss: 2.096 | Acc: 23.942% (6880/28736)


Loss: 2.022 | Acc: 26.099% (10523/40320)
Loss: 2.022 | Acc: 26.117% (10547/40384)
Loss: 2.022 | Acc: 26.115% (10563/40448)
Loss: 2.022 | Acc: 26.121% (10582/40512)
Loss: 2.021 | Acc: 26.131% (10603/40576)
Loss: 2.021 | Acc: 26.152% (10628/40640)
Loss: 2.021 | Acc: 26.155% (10646/40704)
Loss: 2.020 | Acc: 26.170% (10669/40768)
Loss: 2.020 | Acc: 26.190% (10694/40832)
Loss: 2.019 | Acc: 26.215% (10721/40896)
Loss: 2.019 | Acc: 26.223% (10741/40960)
Loss: 2.019 | Acc: 26.229% (10760/41024)
Loss: 2.019 | Acc: 26.232% (10778/41088)
Loss: 2.018 | Acc: 26.247% (10801/41152)
Loss: 2.018 | Acc: 26.262% (10824/41216)
Loss: 2.018 | Acc: 26.267% (10843/41280)
Loss: 2.017 | Acc: 26.267% (10860/41344)
Loss: 2.017 | Acc: 26.270% (10878/41408)
Loss: 2.017 | Acc: 26.271% (10895/41472)
Loss: 2.017 | Acc: 26.262% (10908/41536)
Loss: 2.017 | Acc: 26.262% (10925/41600)
Loss: 2.017 | Acc: 26.267% (10944/41664)
Loss: 2.017 | Acc: 26.263% (10959/41728)
Loss: 2.016 | Acc: 26.261% (10975/41792)
Loss: 2.016 | Ac

Loss: 1.782 | Acc: 33.729% (1619/4800)
Loss: 1.780 | Acc: 33.758% (1642/4864)
Loss: 1.780 | Acc: 33.787% (1665/4928)
Loss: 1.779 | Acc: 33.734% (1684/4992)
Loss: 1.779 | Acc: 33.683% (1703/5056)
Loss: 1.780 | Acc: 33.711% (1726/5120)
Loss: 1.780 | Acc: 33.758% (1750/5184)
Loss: 1.781 | Acc: 33.689% (1768/5248)
Loss: 1.780 | Acc: 33.735% (1792/5312)
Loss: 1.782 | Acc: 33.575% (1805/5376)
Loss: 1.783 | Acc: 33.621% (1829/5440)
Loss: 1.783 | Acc: 33.576% (1848/5504)
Loss: 1.784 | Acc: 33.495% (1865/5568)
Loss: 1.785 | Acc: 33.452% (1884/5632)
Loss: 1.786 | Acc: 33.357% (1900/5696)
Loss: 1.784 | Acc: 33.490% (1929/5760)
Loss: 1.783 | Acc: 33.448% (1948/5824)
Loss: 1.784 | Acc: 33.390% (1966/5888)
Loss: 1.783 | Acc: 33.468% (1992/5952)
Loss: 1.783 | Acc: 33.527% (2017/6016)
Loss: 1.784 | Acc: 33.503% (2037/6080)
Loss: 1.784 | Acc: 33.512% (2059/6144)
Loss: 1.784 | Acc: 33.505% (2080/6208)
Loss: 1.786 | Acc: 33.482% (2100/6272)
Loss: 1.786 | Acc: 33.475% (2121/6336)
Loss: 1.786 | Acc: 33.484

...

Loss: 1.751 | Acc: 34.968% (649/1856)
Loss: 1.752 | Acc: 34.948% (671/1920)
Loss: 1.759 | Acc: 34.526% (685/1984)
Loss: 1.760 | Acc: 34.521% (707/2048)
Loss: 1.762 | Acc: 34.564% (730/2112)
Loss: 1.768 | Acc: 34.467% (750/2176)
Loss: 1.766 | Acc: 34.732% (778/2240)
Loss: 1.764 | Acc: 34.896% (804/2304)
Loss: 1.762 | Acc: 34.840% (825/2368)
Loss: 1.763 | Acc: 34.827% (847/2432)
Loss: 1.762 | Acc: 34.976% (873/2496)
Loss: 1.758 | Acc: 35.078% (898/2560)
Loss: 1.758 | Acc: 34.985% (918/2624)
Loss: 1.758 | Acc: 35.082% (943/2688)
Loss: 1.756 | Acc: 35.138% (967/2752)
Loss: 1.757 | Acc: 34.908% (983/2816)
Loss: 1.755 | Acc: 34.549% (995/2880)
Loss: 1.759 | Acc: 34.341% (1011/2944)
Loss: 1.753 | Acc: 34.475% (1037/3008)
Loss: 1.753 | Acc: 34.505% (1060/3072)
Loss: 1.755 | Acc: 34.534% (1083/3136)
Loss: 1.754 | Acc: 34.438% (1102/3200)
Loss: 1.750 | Acc: 34.344% (1121/3264)
Loss: 1.750 | Acc: 34.345% (1143/3328)
Loss: 1.749 | Acc: 34.257% (1162/3392)
Loss: 1.750 | Acc: 34.288% (1185/3456)
Los

Loss: 1.745 | Acc: 35.253% (5460/15488)
Loss: 1.745 | Acc: 35.275% (5486/15552)
Loss: 1.744 | Acc: 35.304% (5513/15616)
Loss: 1.744 | Acc: 35.338% (5541/15680)
Loss: 1.744 | Acc: 35.309% (5559/15744)
Loss: 1.745 | Acc: 35.318% (5583/15808)
Loss: 1.746 | Acc: 35.301% (5603/15872)
Loss: 1.746 | Acc: 35.316% (5628/15936)
Loss: 1.746 | Acc: 35.319% (5651/16000)
Loss: 1.747 | Acc: 35.296% (5670/16064)
Loss: 1.748 | Acc: 35.280% (5690/16128)
Loss: 1.748 | Acc: 35.308% (5717/16192)
Loss: 1.747 | Acc: 35.322% (5742/16256)
Loss: 1.747 | Acc: 35.331% (5766/16320)
Loss: 1.746 | Acc: 35.370% (5795/16384)
Loss: 1.746 | Acc: 35.372% (5818/16448)
Loss: 1.745 | Acc: 35.386% (5843/16512)
Loss: 1.746 | Acc: 35.358% (5861/16576)
Loss: 1.745 | Acc: 35.379% (5887/16640)
Loss: 1.746 | Acc: 35.351% (5905/16704)
Loss: 1.746 | Acc: 35.353% (5928/16768)
Loss: 1.746 | Acc: 35.373% (5954/16832)
Loss: 1.746 | Acc: 35.381% (5978/16896)
Loss: 1.746 | Acc: 35.401% (6004/16960)
Loss: 1.745 | Acc: 35.409% (6028/17024)


Loss: 1.737 | Acc: 35.778% (10350/28928)
Loss: 1.737 | Acc: 35.755% (10366/28992)
Loss: 1.737 | Acc: 35.769% (10393/29056)
Loss: 1.737 | Acc: 35.749% (10410/29120)
Loss: 1.737 | Acc: 35.746% (10432/29184)
Loss: 1.737 | Acc: 35.726% (10449/29248)
Loss: 1.737 | Acc: 35.743% (10477/29312)
Loss: 1.736 | Acc: 35.778% (10510/29376)
Loss: 1.736 | Acc: 35.788% (10536/29440)
Loss: 1.736 | Acc: 35.782% (10557/29504)
Loss: 1.736 | Acc: 35.789% (10582/29568)
Loss: 1.736 | Acc: 35.786% (10604/29632)
Loss: 1.736 | Acc: 35.766% (10621/29696)
Loss: 1.736 | Acc: 35.746% (10638/29760)
Loss: 1.736 | Acc: 35.740% (10659/29824)
Loss: 1.736 | Acc: 35.740% (10682/29888)
Loss: 1.737 | Acc: 35.710% (10696/29952)
Loss: 1.737 | Acc: 35.714% (10720/30016)
Loss: 1.737 | Acc: 35.695% (10737/30080)
Loss: 1.737 | Acc: 35.679% (10755/30144)
Loss: 1.737 | Acc: 35.683% (10779/30208)
Loss: 1.737 | Acc: 35.683% (10802/30272)
Loss: 1.737 | Acc: 35.694% (10828/30336)
Loss: 1.737 | Acc: 35.704% (10854/30400)
Loss: 1.737 | Ac

Loss: 1.739 | Acc: 35.808% (14988/41856)
Loss: 1.739 | Acc: 35.809% (15011/41920)
Loss: 1.740 | Acc: 35.795% (15028/41984)
Loss: 1.739 | Acc: 35.788% (15048/42048)
Loss: 1.739 | Acc: 35.805% (15078/42112)
Loss: 1.739 | Acc: 35.817% (15106/42176)
Loss: 1.739 | Acc: 35.826% (15133/42240)
Loss: 1.739 | Acc: 35.807% (15148/42304)
Loss: 1.739 | Acc: 35.812% (15173/42368)
Loss: 1.739 | Acc: 35.820% (15199/42432)
Loss: 1.739 | Acc: 35.827% (15225/42496)
Loss: 1.739 | Acc: 35.818% (15244/42560)
Loss: 1.739 | Acc: 35.818% (15267/42624)
Loss: 1.739 | Acc: 35.816% (15289/42688)
Loss: 1.739 | Acc: 35.821% (15314/42752)
Loss: 1.740 | Acc: 35.809% (15332/42816)
Loss: 1.740 | Acc: 35.795% (15349/42880)
Loss: 1.740 | Acc: 35.800% (15374/42944)
Loss: 1.740 | Acc: 35.814% (15403/43008)
Loss: 1.740 | Acc: 35.814% (15426/43072)
Loss: 1.739 | Acc: 35.835% (15458/43136)
Loss: 1.739 | Acc: 35.833% (15480/43200)
Loss: 1.739 | Acc: 35.843% (15507/43264)
Loss: 1.739 | Acc: 35.847% (15532/43328)
Loss: 1.739 | Ac

Loss: 1.785 | Acc: 35.019% (2174/6208)
Loss: 1.785 | Acc: 35.029% (2197/6272)
Loss: 1.786 | Acc: 35.006% (2218/6336)
Loss: 1.786 | Acc: 35.031% (2242/6400)
Loss: 1.787 | Acc: 34.901% (2256/6464)
Loss: 1.787 | Acc: 35.003% (2285/6528)
Loss: 1.788 | Acc: 34.982% (2306/6592)
Loss: 1.788 | Acc: 35.006% (2330/6656)
Loss: 1.787 | Acc: 35.000% (2352/6720)
Loss: 1.787 | Acc: 34.950% (2371/6784)
Loss: 1.786 | Acc: 34.988% (2396/6848)
Loss: 1.788 | Acc: 34.910% (2413/6912)
Loss: 1.788 | Acc: 34.948% (2438/6976)
Loss: 1.789 | Acc: 34.858% (2454/7040)
Loss: 1.790 | Acc: 34.783% (2471/7104)
Loss: 1.788 | Acc: 34.863% (2499/7168)
Loss: 1.790 | Acc: 34.776% (2515/7232)
Loss: 1.789 | Acc: 34.745% (2535/7296)
Loss: 1.788 | Acc: 34.823% (2563/7360)
Loss: 1.787 | Acc: 34.860% (2588/7424)
Loss: 1.785 | Acc: 34.976% (2619/7488)
Loss: 1.785 | Acc: 34.958% (2640/7552)
Loss: 1.787 | Acc: 34.887% (2657/7616)
Loss: 1.786 | Acc: 34.883% (2679/7680)
Loss: 1.785 | Acc: 34.956% (2707/7744)
Loss: 1.784 | Acc: 35.041